# Performance comparison between `RADEX` and `pythonradex`

In this notebook, we want to compare the performance (i.e. execution speed) of `RADEX` and `pythonradex`.

## Initial remarks

- The performance depends on your computing environment (type of CPU etc.).
- The performance also depends on the molecule you consider. This is because different molecules have different numbers of levels and transitions.
- Both codes are single-threaded.

## Preparation

First, some preparation work is needed in order to use `RADEX`. Proceed as follows:
1. Download `RADEX` from the [official RADEX page](https://sronpersonalpages.nl/~vdtak/radex/index.shtml) and install it according to the instructions. Make sure to choose "uniform sphere" as the geometry for the escape probability in `radex.inc`, as we will use that geometry in this example.
2. Download the file `RADEX_wrapper.py` from [here](https://github.com/gica3618/pythonradex/tree/master/docs/notebooks) and place it into the same folder as this notebook. In that file, modify the `radex_path` to point to your `RADEX` executable.

In [1]:
from scipy import constants
import time
import os
import numpy as np
from pythonradex import helpers, radiative_transfer
import RADEX_wrapper

Define the path to the folder that contains your LAMDA-formatted files:

In [2]:
LAMDA_folder = "/home/gianni/science/LAMDA_database_files"

In this example, we consider CO, which has an intermediate number of levels and transitions.

## Single execution

Ultimately, we want to compare the performance of `RADEX` and `pythonradex` to compute a grid of models. However, it is useful to first consider a single execution to understand which steps of the execution take most of the time.

In [3]:
datafilename = "co.dat"
collider_densities = {"ortho-H2": 1e4 * constants.centi**-3}
# assume thermal ortho/para ratio
collider_densities["para-H2"] = collider_densities["ortho-H2"] / 3
Tkin = 75
T_background = 2.73
N = 1e17 * constants.centi**-2
width_v = 3 * constants.kilo
# be careful with modifying the geometry. If you change the geometry for pythonradex,
# you also need to re-compile RADEX with the new geometry
geometry = "static sphere"
line_profile_type = "Gaussian"


def ext_background(nu):
    return helpers.B_nu(T=T_background, nu=nu)


datafilepath = os.path.join(LAMDA_folder, datafilename)

In [4]:
def timer(func, operation_name, args=(), kwargs={}):
    start = time.time()
    output = func(*args, **kwargs)
    end = time.time()
    duration = end - start
    print(f"{operation_name}: {duration:.2g} s")
    return output, duration

### `RADEX`

We check how long it takes to
1. Write the input file
2. Run `RADEX`
3. Read the results from the output file

In [5]:
input_filepath = "test.inp"
output_filepath = "test.out"

write_kwargs = {
    "datafilename": datafilename,
    "collider_densities": collider_densities,
    "Tkin": Tkin,
    "T_background": T_background,
    "N": N,
    "width_v": width_v,
    "input_filepath": input_filepath,
    "output_filepath": output_filepath,
}
_, input_time = timer(
    func=RADEX_wrapper.write_radex_input_file,
    kwargs=write_kwargs,
    operation_name="write RADEX input",
)
_, run_time = timer(
    func=RADEX_wrapper.run_radex,
    kwargs={"input_filepath": input_filepath},
    operation_name="run RADEX",
)
_, read_time = timer(
    func=RADEX_wrapper.read_radex_output,
    kwargs={"output_filepath": output_filepath},
    operation_name="read RADEX output",
)
total = input_time + run_time + read_time
print(f"total: {total:.2g} s\n")
single_execution_time_RADEX = total

write RADEX input: 0.0014 s
run RADEX: 0.076 s
read RADEX output: 0.0037 s
total: 0.081 s



We see that writing the input file and reading the output file is negligible compared to running `RADEX`.

### `pythonradex`

We check how long it takes to
1. Initialise
2. Update the parameters
3. Solve the radiative transfer
4. Retrieve exitation temperatures and optical depths
5. Calculate brightness temperatures 
6. Calculate total fluxes

In [7]:
source_kwargs = {
    "datafilepath": datafilepath,
    "geometry": geometry,
    "line_profile_type": line_profile_type,
    "width_v": width_v,
}
update_kwargs = {
    "ext_background": ext_background,
    "N": N,
    "Tkin": Tkin,
    "collider_densities": collider_densities,
    "T_dust": 0,
    "tau_dust": 0,
}
source, init_time = timer(
    func=radiative_transfer.Source,
    kwargs=source_kwargs,
    operation_name="initialisation",
)
_, update_time = timer(
    func=source.update_parameters,
    kwargs=update_kwargs,
    operation_name="update parameters",
)
_, solve_time = timer(
    func=source.solve_radiative_transfer,
    kwargs={},
    operation_name="solve radiative transfer",
)
_, Tex_time = timer(
    func=getattr, args=(source, "Tex"), operation_name="retrieve excitation temperature"
)
_, tau_time = timer(
    func=getattr,
    args=(source, "tau_nu0_individual_transitions"),
    operation_name="retrieve optical depth",
)
_, Tbrightness_time = timer(
    func=source.emission_at_line_center,
    kwargs={"output_type": "Rayleigh-Jeans"},
    operation_name="calculate brightness temperatures",
)
_, flux_time = timer(
    func=source.frequency_integrated_emission,
    kwargs={"output_type": "flux", "solid_angle": 1, "transitions": None},
    operation_name="calculate fluxes",
)
total = (
    init_time
    + update_time
    + solve_time
    + Tex_time
    + tau_time
    + Tbrightness_time
    + flux_time
)
print(f"total: {total:.2g} s\n")
print(f"RADEX/pythonradex: {single_execution_time_RADEX/total:.2g}")

initialisation: 0.24 s
update parameters: 0.0012 s
solve radiative transfer: 0.0068 s
retrieve excitation temperature: 2.1e-06 s
retrieve optical depth: 2.1e-06 s
calculate brightness temperatures: 0.0002 s
calculate fluxes: 0.0012 s
total: 0.25 s

RADEX/pythonradex: 0.32


You may notice that when running the above cell for the first time, the execution can take several seconds. When running the cell again, it is much faster. This is because during the first run, parts of the code are compiled using the `numba` package.

Overall, we see that for `pythonradex`, initialisation is most expensive. The time it takes to update parameters and calculate brightness temperatures or fluxes is almost negligible.

Comparing the execution times, `RADEX` is clearly faster. However, if we calculate many models, `pythonradex` should be faster since initialisation only needs to be done once. This is verified in the next section.

## Multiple executions

Let's calculate the same model many times and compare the performance.

In [8]:
n_calculations = 1000

### `RADEX`

In [9]:
start = time.time()
for _ in range(n_calculations):
    # following function writes input, executes RADEX and reads the output:
    RADEX_wrapper.run(
        datafilename=datafilename,
        collider_densities=collider_densities,
        Tkin=Tkin,
        T_background=T_background,
        N=N,
        width_v=width_v,
        input_filepath=input_filepath,
        output_filepath=output_filepath,
    )
end = time.time()
duration = end - start
radex_time_multi_exec = duration
print(f"RADEX ({n_calculations} calculations): {duration:.2g} s")
print(
    f"(expected from single execution: {n_calculations*single_execution_time_RADEX:.2g} s)\n"
)

RADEX (1000 calculations): 47 s
(expected from single execution: 81 s)



The time is very roughly the number of models multiplied by the time it takes for a single execution.

### `pythonradex`

In [10]:
start = time.time()
source = radiative_transfer.Source(**source_kwargs)
for _ in range(n_calculations):
    source.update_parameters(**update_kwargs)
    source.solve_radiative_transfer()
    # retrieve Tex and tau:
    source.Tex
    source.tau_nu0_individual_transitions
    # calculate brightness temperature:
    source.emission_at_line_center(output_type="Rayleigh-Jeans")
    # calculate frequency-integrated emission:
    source.frequency_integrated_emission(
        output_type="flux", solid_angle=1, transitions=None
    )
end = time.time()
duration = end - start
print(f"pythonradex ({n_calculations} calculations): {duration:.2g} s")
print(f"time ratio RADEX/pythonradex: {radex_time_multi_exec/duration:.2g}")

pythonradex (1000 calculations): 6.3 s
time ratio RADEX/pythonradex: 7.4


Now `pythonradex` is faster by a factor of roughly 7.

## Model grid

Finally, let's compare the performance when calculating a grid of models.

In [11]:
nH2_grid = np.logspace(3, 7, 10) * constants.centi**-3
Tkin_grid = np.logspace(1, np.log10(300), 10)
N_grid = np.logspace(13, 18, 10) * constants.centi**-2


def get_collider_densities(nH2):
    # assume thermal ortho/para ratio
    return {"ortho-H2": nH2 * 3 / 4, "para-H2": nH2 / 4}

### `RADEX`

In [12]:
start = time.time()
for nH2 in nH2_grid:
    collider_densities = get_collider_densities(nH2=nH2)
    for Tkin in Tkin_grid:
        for N in N_grid:
            RADEX_wrapper.run(
                datafilename=datafilename,
                collider_densities=collider_densities,
                Tkin=Tkin,
                T_background=T_background,
                N=N,
                width_v=width_v,
                input_filepath=input_filepath,
                output_filepath=output_filepath,
            )
end = time.time()
radex_grid_time = end - start
print(f"grid calculation (RADEX): {radex_grid_time:.2g} s")

Note: The following floating-point exceptions are signalling: IEEE_INVALID_FLAG
Note: The following floating-point exceptions are signalling: IEEE_INVALID_FLAG
Note: The following floating-point exceptions are signalling: IEEE_INVALID_FLAG
Note: The following floating-point exceptions are signalling: IEEE_INVALID_FLAG
Note: The following floating-point exceptions are signalling: IEEE_INVALID_FLAG
Note: The following floating-point exceptions are signalling: IEEE_INVALID_FLAG
Note: The following floating-point exceptions are signalling: IEEE_INVALID_FLAG
Note: The following floating-point exceptions are signalling: IEEE_INVALID_FLAG
Note: The following floating-point exceptions are signalling: IEEE_INVALID_FLAG
Note: The following floating-point exceptions are signalling: IEEE_INVALID_FLAG
Note: The following floating-point exceptions are signalling: IEEE_INVALID_FLAG
Note: The following floating-point exceptions are signalling: IEEE_INVALID_FLAG
Note: The following floating-point excep

grid calculation (RADEX): 53 s


### `pythonradex`

We optimize the performance by putting column density in the innermost loop, since updating column density is less expensive compared to collider density or kinetic temperature. However, one may verify that in this particular case, it doesn't change anything (i.e. putting column density in the outermost loop does not affect performance negatively). Noet that the method `update_parameters` only updates those parameters that are different from the current parameters.

In [13]:
start = time.time()
source = radiative_transfer.Source(**source_kwargs, warn_negative_tau=False)
# update the parameters outside the loop so that
# inside the loop we don't need to update ext_background,
# T_dust and tau_dust anymore (in this particular case, it does not change
# the performance though)
source.update_parameters(**update_kwargs)

for nH2 in nH2_grid:
    collider_densities = get_collider_densities(nH2=nH2)
    for Tkin in Tkin_grid:
        for N in N_grid:
            source.update_parameters(
                collider_densities=collider_densities, Tkin=Tkin, N=N
            )
            source.solve_radiative_transfer()
            source.frequency_integrated_emission(
                output_type="flux", solid_angle=1, transitions=None
            )
end = time.time()
duration = end - start
print(f"grid calculation (RADEX): {duration:.2g} s")
print(f"grid calculation time ratio RADEX/pythonradex: {radex_grid_time/duration:.3g}")

grid calculation (RADEX): 5.7 s
grid calculation time ratio RADEX/pythonradex: 9.45


`pythonradex` calculates the grid roughly 10 times faster than `RADEX`.